In [37]:
import os
import requests
import json
import openai

import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

In [38]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [39]:
df_MAF=pd.read_csv('data/MAF_Evaluation.csv')

In [40]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')

In [41]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [42]:
maf=df_MAF['item'].to_list()

In [43]:
len(maf)

1876

In [44]:
call=iab_labels+maf

In [45]:
len(call)

2036

In [46]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [47]:
total_embeddings=[]

In [48]:
def create_embeddings(item,total_embeddings):
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

    

In [49]:
for items in tqdm(call):
    create_embeddings(items,total_embeddings)

100%|██████████| 2036/2036 [05:10<00:00,  6.57it/s]


In [50]:
len(total_embeddings)

2036

In [51]:
iab_embeddings=total_embeddings[0:160]

In [52]:
maf_embeddings=total_embeddings[160:]

In [53]:
def calculate_precision_recall(true_labels, predictions):
    true_positives = sum(1 for pred in predictions if pred in true_labels)
    false_positives = len(predictions) - true_positives
    false_negatives = len(true_labels) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    
    return precision, recall

In [54]:
def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [55]:
nodes=df_MAF['item'].to_list()
#node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
#H_Mappings=df_MAF['H_Scores'].to_list()

In [56]:
cos_sim=cosine_similarity(np.array(iab_embeddings[90]).reshape(1,-1),np.array(maf_embeddings[15]).reshape(1,-1))
print(cos_sim)

[[0.77228327]]


In [106]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_node_ids=[]
top_n_IAB_Candidates=[]
top_n_H_scores=[]
top_n_labels=[]
top_n_cosines=[]

for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)


    #threshold logic needed
    similarities=[x for x in similarities if x[2]>=0.765]
    #similarities=similarities[0:55]

    
    
    if len(similarities)==0:
        similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in similarities:
        #print(items[1])
        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])
        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])
        #top_n_node_ids.append(node_id[i])
        top_n_IAB_Candidates.append(IAB_Candidates[i])
        #top_n_H_scores.append(H_Mappings[i])


    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)


100%|██████████| 1876/1876 [02:22<00:00, 13.16it/s]


In [107]:
len(top_n_labels)

1876

In [108]:
len(top_n_labels_new)

122829

In [109]:
len(top_n_cosines)

1876

In [110]:
MAF_predictions=df_MAF['IAB_Candidates'].to_list()

In [62]:
# sanity_check=[]
# for _ in range(len(MAF_predictions)):
#     sanity_check.append(iab_labels)

In [63]:
# candidate_count=0
# hit_count=0
# precision=0
# recall=0
# count=0
# f1=0

# for predictions, labels in zip(MAF_predictions,sanity_check):
#     count+=1
#     proper=ast.literal_eval(predictions)
#     #prec, reca=calculate_precision_recall(predictions, labels)

#     #f=calculate_f1_score(prec, reca)
#     #precision+=prec
#     #recall+=reca
#     #f1+=f


#     candidate_count+=len(proper)
   
#     for items in proper:
#         if items in labels:
#             hit_count+=1
    
#     print(candidate_count)
#     print(hit_count)


In [111]:
top_n_labels[0]

['Birds',
 'Commercial Trucks',
 'Cruises',
 'Pickup Trucks',
 'Convertible',
 'Cats',
 'SUV',
 'Coupe',
 'Minivan',
 'Travel',
 'Camping',
 'Large Animals',
 'Sports',
 'Beauty',
 'Movies',
 'Graduation',
 'Science',
 'Motorcycles',
 'Laptops',
 'Dating',
 'Musical Instruments',
 'Desktops',
 'Dogs',
 'Auto Type',
 'Rail Travel',
 'Automotive',
 'Station Wagon',
 'Air Travel',
 'Cameras and Camcorders',
 'Houses',
 'Wedding',
 'Cooking',
 'Television',
 'Green Vehicles',
 'Bath and Shower',
 'Sedan',
 'Shopping',
 'Smartphones',
 'Outdoor Activities',
 'Reptiles',
 'Grocery Shopping',
 'Landscaping',
 'Prom',
 'Barbecues and Grilling',
 'Education',
 'Auto Repair',
 'Aviation Industry',
 'Home Appliances',
 'Parks & Nature',
 'Games and Puzzles',
 'Technology & Computing',
 'Land and Farms',
 'Gardening',
 'Auto Rentals',
 "Children's Games and Toys",
 'Wellness',
 'Space and Astronomy',
 'Auto Parts',
 'Apartments',
 'Parenting',
 'Video Gaming',
 'Road-Side Assistance',
 'Remodeling

In [112]:
candidate_count=0
hit_count=0
precision=0
recall=0
count=0
f1=0

for predictions, labels in zip(MAF_predictions,top_n_labels):
    count+=1
    proper=ast.literal_eval(predictions)
    prec, reca=calculate_precision_recall(predictions, labels)

    f=calculate_f1_score(prec, reca)
    precision+=prec
    recall+=reca
    f1+=f


    candidate_count+=len(proper)
    # for items in labels:
    #     if items in proper:
    #         hit_count+=1

    for items in proper:
        if items in labels:
            hit_count+=1

In [113]:
hit_count/candidate_count

0.8940686371751636

In [114]:
precision/942

0.0779996253797869

In [115]:
recall/942

0.12264216664289772

In [116]:
f1/942

0.07903320271290601

In [117]:
dic={'item':top_n_nodes, 'IAB_Candidates': top_n_IAB_Candidates,
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [118]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [119]:
df_MAF_new.to_csv('results/OpenAI_results_0.765_newmapping.csv')

In [120]:
#Average adwords per item
count=0
for items in tqdm(maf):
    count+=df_MAF_new[df_MAF_new['item']==items].shape[0]

print(count/len(maf))

100%|██████████| 1876/1876 [00:10<00:00, 174.52it/s]

65.47388059701493


In [121]:
counts=[]
for items in tqdm(iab_labels):
    lol=df_MAF_new[df_MAF_new['predictions']==items].shape[0]
    counts.append(lol)

dic={'IAB_Label':iab_labels, 'Count':counts}

df_adwords=pd.DataFrame.from_dict(dic)
df_adwords.to_csv('results/Adwords_Statistics.csv',index=False)

100%|██████████| 160/160 [00:00<00:00, 175.17it/s]


In [122]:
iab_labels[counts.index(min(counts))]

'Financial Industry'

In [123]:
sum(counts)/160

767.68125

In [124]:
print(count/len(maf))

65.47388059701493


In [78]:
df_MAF_new[df_MAF_new['item']=='crane'].shape[0]

113

In [79]:
df_MAF_new.shape

(122829, 4)

In [80]:
#Top 100 in predictions but not in embedding

In [81]:
df_MAF_new=df_MAF_new.sort_values(by='cosine_predicted', ascending=False)

In [82]:
df_MAF_new.iloc[0]

item                                                       Television
IAB_Candidates      ['Entertainment Industry', 'Media Industry', '...
predictions                                                Television
cosine_predicted                                 [1.0000000000000004]
Name: 8084, dtype: object

In [83]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [84]:
items=df_MAF_new['item'].to_list()
IAB_Candidates=df_MAF_new['IAB_Candidates'].to_list()
predictions=df_MAF_new['predictions'].to_list()
cosine_predicted=df_MAF_new['cosine_predicted'].to_list()
H_Mapping_scores=df_MAF_new['H_Mapping_scores'].to_list()

KeyError: 'H_Mapping_scores'

In [ ]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    pred_list=ast.literal_eval(IAB)
    if pred not in pred_list:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

253it [00:00, 19364.57it/s]


In [ ]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [ ]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [ ]:
df_top_100.to_excel('results/OpenAI_Top_100_Not_In_H.xlsx', index=False)

In [ ]:
#Top 100 in H but not in Embedding

In [ ]:
print(top_n_labels[5])
print(top_n_cosines[5])

['Dogs', 'SUV', 'Cats', 'Large Animals', 'Sedan', 'Pets', 'Polar Travel', 'Camping', 'Beauty', 'Desktops', 'Wellness', 'Science', 'Minivan', 'Reptiles', 'Pickup Trucks', 'Laptops', 'Sports', 'Travel', 'Outdoor Activities', 'Hair Care', 'Hobbies & Interests', 'Birds', 'Cruises', 'Parenting', 'Dating', 'Convertible', 'Games and Puzzles', 'Movies', 'Shaving', 'Smartphones', 'Motorcycles', 'Shopping', 'Graduation', 'Cooking', 'Parks & Nature', 'Holiday Shopping', 'Space and Astronomy', 'Luxury Cars', 'Healthy Living', 'Single Life', 'Education', 'Commercial Trucks', 'Rail Travel', 'Houses', 'Station Wagon', 'Wedding', 'Travel Locations', "Women's Handbags and Wallets", 'Coupe', 'Travel Accessories', 'Beach Travel', 'Apartments', 'Grocery Shopping', 'Asia Travel', 'South America Travel', 'Board Games and Puzzles', 'Barbecues and Grilling', 'Bath and Shower', 'Europe Travel', 'Landscaping', 'Fitness and Exercise', 'Home Security', 'Smart Home', 'Air Travel', 'Home & Garden', 'Gardening', 'Zo

In [ ]:
nodes=df_MAF['item'].to_list()
# node_id=df_MAF['node_id'].to_list()
IAB_Candidates=df_MAF['IAB_Candidates'].to_list()
H_Mappings=df_MAF['H_Scores'].to_list()

In [ ]:
mid_node=[]
mid_IAB=[]
mid_label=[]
mid_cos=[]
mid_H_map=[]

for node, IAB, label, cos, H_map in tqdm(zip(nodes,IAB_Candidates,top_n_labels,top_n_cosines,H_Mappings)):
    IAB_list=ast.literal_eval(IAB)
    H_Map_list=ast.literal_eval(H_map)
    for IAB_item, H_Map_item in zip(IAB_list,H_Map_list):
        mid_node.append(node)
        mid_IAB.append(IAB_item)
        mid_label.append(label)
        mid_cos.append(cos)
        mid_H_map.append(H_Map_item)

942it [00:00, 34295.68it/s]


In [ ]:
dic={'item':mid_node, 'IAB_Candidates': mid_IAB,
'predictions': mid_label, 'cosine_predicted': mid_cos, "H_Mapping_scores": mid_H_map}

In [ ]:
df_lol_100=pd.DataFrame.from_dict(dic)

In [ ]:
df_lol_100=df_lol_100.sort_values(by='H_Mapping_scores', ascending=False)

In [ ]:
df_lol_100.head()

,item,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores
2613,wagon,Travel,"[Station Wagon, Minivan, SUV, Convertible, Tra...","[[0.8676690064930836], [0.8482987892336509], [...",1.0
2614,drop,Real Estate,"[Dating, Travel, Graduation, Movies, Shopping,...","[[0.8176049105530151], [0.8087438873386152], [...",1.0
2615,drop,Travel,"[Dating, Travel, Graduation, Movies, Shopping,...","[[0.8176049105530151], [0.8087438873386152], [...",1.0
2629,mousetrap,Home & Garden,"[Cats, Movies, Coupe, Birds, Games and Puzzles...","[[0.7973278023641381], [0.7922019973205039], [...",1.0
2626,ring_mail,Events and Attractions,"[Rail Travel, Travel, Birds, Nail Care, Hair C...","[[0.7915800550643299], [0.7658795812872278], [...",1.0


In [ ]:
top_100_node=[]
top_100_embedding_pred=[]
top_100_H_pred=[]
top_100_cos_pred=[]
top_100_H_score=[]

In [ ]:
items=df_lol_100['item'].to_list()
IAB_Candidates=df_lol_100['IAB_Candidates'].to_list()
predictions=df_lol_100['predictions'].to_list()
cosine_predicted=df_lol_100['cosine_predicted'].to_list()
H_Mapping_scores=df_lol_100['H_Mapping_scores'].to_list()

NameError: name 'df_lol_100' is not defined

In [ ]:
for item,IAB,pred,cos,H_map in tqdm(zip(items,IAB_Candidates,predictions,cosine_predicted,H_Mapping_scores)):
    #pred_list=ast.literal_eval(pred)
    if IAB not in pred:
        top_100_node.append(item)
        top_100_embedding_pred.append(pred)
        top_100_H_pred.append(IAB)
        top_100_cos_pred.append(cos)
        top_100_H_score.append(H_map)
    if len(top_100_node)==100:
        break

1311it [00:00, 678855.87it/s]


In [ ]:
top_100_dic={'items':top_100_node, 'Embedding Predictions': top_100_embedding_pred,
 "H_Mapping_Pred": top_100_H_pred, 'Predicted Cosine': top_100_cos_pred, 
 'H Mapping Cosine': top_100_H_score}

In [ ]:
df_top_100=pd.DataFrame.from_dict(top_100_dic)

In [ ]:
df_top_100.to_excel('results/Top_100_Not_In_OpenAI.xlsx', index=False)